# Load Ntuple and DecayHash

In [1]:
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import sys
sys.path.append('/home/belle/zhangboy/inclusive_R_D/')
import utilities as util
import decayHash
from decayHash import DecayHashMap

Welcome to JupyROOT 6.24/06


In [27]:
# read a signal mc as a pandas dataframe
files = ['MC14ri_sigDDst_foldex_e_3/sigDDst_0.parquet', 
         'MC14ri_normDDst_foldex_e_5/normDDst_0.parquet',
         'MC14ri_Dststell2_foldex_e_2/Dststell2_0.parquet', 
         'MC14ri_DststTau1_foldex_e_2/DststTau1_0.parquet',
         'MC14ri_DststTau2_foldex_e_2/DststTau2_0.parquet']

filename = f'../Samples/Signal_MC14ri/{files[0]}'
decayhash = f'../Samples/Signal_MC14ri/MC14ri_sigDDst_foldex_e_3/hashmap_sigDDst_e.root'
# filename=f'../Samples/Signal_MC_ROEx1/B2D_l_nu/Ntuples/MC_e_50k_0.parquet'
# decayhash=f'../Samples/Signal_MC_ROEx1/B2D_l_nu/Ntuples/hashmap_MC_e_50k.root'
hashmap1 = DecayHashMap(decayhash, removeRadiativeGammaFlag=False)
#hashmap2 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)
data = pd.read_parquet(filename, engine="pyarrow")
df, samples=util.get_dataframe_samples(data, 'e')

for name in samples:
    print(f'{name}: {len(samples[name])}')

$D\tau\nu$: 0
$D^\ast\tau\nu$: 0
$D^{\ast\ast}\tau\nu$: 109694
$D\ell\nu$: 0
$D^\ast\ell\nu$: 0
$D^{\ast\ast}\ell\nu$: 15290
bkg_fakeDTC: 1055890
bkg_fakeB: 72922
bkg_continuum: 0
bkg_others: 62


In [ ]:
import pandas
pandas.set_option('display.max_rows', None)
print(data.isna().sum())

# e channel

In [26]:
pd.set_option('display.max_rows', None)
a=samples[r'bkg_others'].query('D_mcErrors>=0')[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','D_mcErrors','B0_mcErrors','e_mcErrors','DecayMode']].value_counts()
a

B0_mcPDG  D_mcPDG  e_mcPDG  e_genMotherPDG  D_mcErrors  B0_mcErrors  e_mcErrors  DecayMode
-511.0     411.0    13.0     15.0           0.0         408.0        384.0       5            15
 511.0    -411.0   -13.0    -15.0           0.0         408.0        384.0       5            12
-511.0     411.0    13.0     15.0           0.0         408.0        2432.0      5             8
 511.0    -411.0   -11.0     511.0          32.0        40.0         0.0         2             5
                   -13.0    -15.0           0.0         408.0        2432.0      5             3
                                                        424.0        384.0       5             3
-511.0     411.0    13.0     15.0           0.0         424.0        384.0       5             2
                                                                     2432.0      5             2
                    11.0    -511.0          32.0        40.0         0.0         2             1
                                    

In [22]:
candidate = samples[r'$D\ell\nu$'].query('D_mcErrors==0 and B0_mcErrors==8').iloc[2]
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values

print(candidate[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','DecayMode','B0_mcErrors','D_mcErrors',
                 'e_mcErrors','e_isBremsCorrected','B0_CMS3_weMissM2']])
# print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
# org2 = hashmap2.get_original_decay(*hash_values)
# print(org2.to_string())

org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

B0_mcPDG                 511.0
D_mcPDG                 -411.0
e_mcPDG                  -11.0
e_genMotherPDG           511.0
DecayMode                    2
B0_mcErrors                8.0
D_mcErrors                 0.0
e_mcErrors              2048.0
e_isBremsCorrected         1.0
B0_CMS3_weMissM2      0.188957
Name: 46, dtype: object
300553
    -511
        413
            421
                -321
                211
            211
        15
            13
            -14
            16
        -16
    511
        -411
            321
            -211
            -211
        -11
        12



In [11]:
df3 = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>4 and p_D_l<2')
df3.eval('MM2 = (0.5*Ecms - D_CMS_E - e_CMS_E)**2 - (D_CMS_px + e_CMS_px)**2 - (D_CMS_py + e_CMS_py)**2 - (D_CMS_pz + e_CMS_pz)**2', inplace=True)
df3.eval('mcMM2 = (0.5*Ecms - D_CMS_mcE - e_CMS_mcE)**2 - (D_CMS_mcPX + e_CMS_mcPX)**2 - (D_CMS_mcPY + e_CMS_mcPY)**2 - (D_CMS_mcPZ + e_CMS_mcPZ)**2', inplace=True)
df3[['B0_mcErrors','e_mcErrors','B0_CMS_mcP','B0_CMS_p','e_CMS_mcP','e_CMS_p','D_CMS_mcP','D_CMS_p','mcMM2','B0_CMS3_weMissM2','p_D_l']]

,D_mcPDG,mu_mcPDG,mu_genMotherPDG,B0_mcErrors,D_mcErrors,mu_mcErrors,B0_CMS3_weMissM2,p_D_l
5424,-411.0,-13.0,511.0,8.0,0.0,0.0,4.415084,1.488679
9489,-411.0,-13.0,511.0,8.0,0.0,0.0,3.340456,1.946793
11970,411.0,13.0,-511.0,8.0,0.0,0.0,4.258790,1.887789
13022,411.0,13.0,-511.0,8.0,0.0,0.0,3.854828,1.695532
13885,411.0,13.0,-511.0,8.0,0.0,0.0,4.319704,1.954827


In [91]:
pd.set_option('display.max_rows', None)
b=df.query('B0_mcErrors==24')[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','D_mcErrors','B0_mcErrors','e_mcErrors','DecayMode']].value_counts()
b

B0_mcPDG  D_mcPDG  e_mcPDG  e_genMotherPDG  D_mcErrors  B0_mcErrors  e_mcErrors  DecayMode
 511.0    -411.0   -11.0     511.0          0.0         24.0         0.0         4            5
-511.0     411.0    11.0    -511.0          0.0         24.0         0.0         4            4
                                                                                 6            3
 511.0    -411.0   -11.0     511.0          0.0         24.0         0.0         6            2
                             431.0          0.0         24.0         0.0         0            1
dtype: int64

In [5]:
candidate = samples[r'$D^\ast\tau\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2').iloc[7]
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values

print(candidate[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','DecayMode','B0_mcErrors','D_mcErrors',
                 'e_mcErrors','e_isBremsCorrected','e_d0_mcPDG','e_d1_mcPDG','B0_CMS3_weMissM2']])
# print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
# org2 = hashmap2.get_original_decay(*hash_values)
# print(org2.to_string())

org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

B0_mcPDG                -511.0
D_mcPDG                  411.0
e_mcPDG                   11.0
e_genMotherPDG          -511.0
DecayMode                    2
B0_mcErrors                8.0
D_mcErrors                 0.0
e_mcErrors                 0.0
e_isBremsCorrected         0.0
e_d0_mcPDG                11.0
e_d1_mcPDG                 NaN
B0_CMS3_weMissM2      6.988806
Name: 12758, dtype: object
300553
    511
        331
            111
                22
                22
            111
                22
                22
            221
                -211
                211
                111
                    22
                    22
        -4312
            -4132
                -211
                211
                -3324
                    -3322
                        -3122
                            -2212
                            211
                        111
                            22
                            22
                    111
            

# Mu Channel

In [29]:
pd.set_option('display.max_rows', None)
a=samples[r'bkg_others'][['B0_mcPDG','D_mcPDG','mu_mcPDG','mu_genMotherPDG','D_mcErrors','B0_mcErrors','mu_mcErrors','DecayMode']].value_counts()
a

B0_mcPDG  D_mcPDG  mu_mcPDG  mu_genMotherPDG  D_mcErrors  B0_mcErrors  mu_mcErrors  DecayMode
-511.0     411.0    13.0     -511.0           0.0         140.0        2180.0       7            6
 511.0    -411.0   -13.0      511.0           0.0         136.0        2176.0       7            6
                                                          140.0        2180.0       7            6
-511.0     411.0    13.0     -511.0           0.0         136.0        2176.0       7            5
                                                                       128.0        7            1
dtype: int64

In [14]:
df3 = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2')
df3[['D_mcPDG','mu_mcPDG','mu_genMotherPDG','B0_mcErrors','D_mcErrors','mu_mcErrors','B0_CMS3_weMissM2','p_D_l']]

,D_mcPDG,mu_mcPDG,mu_genMotherPDG,B0_mcErrors,D_mcErrors,mu_mcErrors,B0_CMS3_weMissM2,p_D_l
5424,-411.0,-13.0,511.0,8.0,0.0,0.0,4.415084,1.488679
9489,-411.0,-13.0,511.0,8.0,0.0,0.0,3.340456,1.946793
11970,411.0,13.0,-511.0,8.0,0.0,0.0,4.258790,1.887789
13022,411.0,13.0,-511.0,8.0,0.0,0.0,3.854828,1.695532
13885,411.0,13.0,-511.0,8.0,0.0,0.0,4.319704,1.954827


In [28]:
df3 = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2')
df3[['D_mcPDG','mu_mcPDG','mu_genMotherPDG','B0_mcErrors','D_mcErrors','mu_mcErrors','B0_CMS3_weMissM2','p_D_l']]

,D_mcPDG,mu_mcPDG,mu_genMotherPDG,B0_mcErrors,D_mcErrors,mu_mcErrors,B0_CMS3_weMissM2,p_D_l
5350,-411.0,-13.0,511.0,8.0,0.0,0.0,4.415084,1.488679
9361,-411.0,-13.0,511.0,8.0,0.0,0.0,3.340456,1.946793
11810,411.0,13.0,-511.0,8.0,0.0,0.0,4.258790,1.887789
12843,411.0,13.0,-511.0,8.0,0.0,0.0,3.854828,1.695532
13694,411.0,13.0,-511.0,8.0,0.0,0.0,4.319704,1.954827


In [23]:
candidate = samples[r'$D\ell\nu$'].query('nElectrons90+nMuons90==1 and B0_CMS3_weMissM2>2 and p_D_l<2').iloc[4]
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values

print(candidate[['B0_mcPDG','D_mcPDG','mu_mcPDG','mu_genMotherPDG','DecayMode','B0_mcErrors','D_mcErrors',
                 'mu_mcErrors','B0_CMS3_weMissM2']])
# print("Monte Carlo Decay with removed Bremsstrahlung gammas: ")
# org2 = hashmap2.get_original_decay(*hash_values)
# print(org2.to_string())

org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

B0_mcPDG              -511.0
D_mcPDG                411.0
mu_mcPDG                13.0
mu_genMotherPDG       -511.0
DecayMode                  7
B0_mcErrors              8.0
D_mcErrors               0.0
mu_mcErrors              0.0
B0_CMS3_weMissM2    4.319704
Name: 13885, dtype: object
300553
    511
        -413
            -421
                -20213
                    113
                        211
                        -211
                    -211
                321
            -211
        223
            -211
            211
            111
                22
                22
        211
        -213
            -211
            111
                22
                22
        213
            211
            111
                22
                22
    -511
        411
            -321
            211
            211
        13
        -14
        22
        22

